In [45]:
import numpy as np
import spacy


### Задание 1

In [46]:
nlp = spacy.load('en_core_web_sm')
word_1 = 'cat'
word_2 = 'cow'

In [47]:
def cosine_dist(vec1, vec2, vectorised=False):
    if vectorised:
        pass
    else:
        vec1 = nlp(vec1).vector
        vec2 = nlp(vec2).vector
    dot_prod = np.dot(vec1, vec2)
    norm_vec2 = np.linalg.norm(vec1)*np.linalg.norm(vec2)
    result = round(dot_prod/norm_vec2, 3)
    
    return result

In [48]:
answer_1 = cosine_dist(word_1, word_2)
print(answer_1)

0.775


### Задание 2

In [ ]:
sent_1 = "Сегодня очень холодно на улице"
sent_2 = "На улице сегодня очень холодно"

In [50]:
answer_2 = cosine_dist(sent_1, sent_2)
print(answer_2)

0.928


### Заданиие 3

In [51]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [52]:
nlp = spacy.load('en_core_web_sm')

In [53]:
data = pd.read_csv("./data/product_description.csv")

In [54]:
data

,product_name,description
0,Chair,This comfortable chair is perfect for any room...
1,Sofa,This beautiful sofa is a great addition to any...
2,Table,This elegant table is perfect for your dining ...
3,Bed,This cozy bed is perfect for a good night's sl...
4,Bookshelf,This modern bookshelf is perfect for displayin...
5,Desk,This functional desk is perfect for your home ...
6,Ottoman,This stylish ottoman is perfect for your livin...
7,Cabinet,This versatile cabinet is perfect for storing ...
8,Dresser,This elegant dresser is perfect for your bedro...
9,TV Stand,This sleek TV stand is perfect for your entert...


In [55]:
def preprocess_text(text):
    doc = nlp(text)
    tokens = [
        token.lemma_.lower()
        for token in doc
        if not token.is_stop and not token.is_punct
    ]
    return " ".join(tokens)


data["processed_text"] = data["description"].apply(preprocess_text)

In [56]:
data

,product_name,description,processed_text
0,Chair,This comfortable chair is perfect for any room...,comfortable chair perfect room home feature so...
1,Sofa,This beautiful sofa is a great addition to any...,beautiful sofa great addition living room slee...
2,Table,This elegant table is perfect for your dining ...,elegant table perfect dining room durable wood...
3,Bed,This cozy bed is perfect for a good night's sl...,cozy bed perfect good night sleep soft mattres...
4,Bookshelf,This modern bookshelf is perfect for displayin...,modern bookshelf perfect display favorite book...
5,Desk,This functional desk is perfect for your home ...,functional desk perfect home office spacious s...
6,Ottoman,This stylish ottoman is perfect for your livin...,stylish ottoman perfect living room bedroom fo...
7,Cabinet,This versatile cabinet is perfect for storing ...,versatile cabinet perfect store belonging mult...
8,Dresser,This elegant dresser is perfect for your bedro...,elegant dresser perfect bedroom spacious drawe...
9,TV Stand,This sleek TV stand is perfect for your entert...,sleek tv stand perfect entertainment center mu...


In [57]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data["processed_text"])

In [58]:
cosine_matrix = cosine_similarity(vectors)

similarity_df = pd.DataFrame(
    cosine_matrix, index=data["product_name"], columns=data["product_name"]
)

# similarity_df.reset_index()
similarity_pairs = []

for i in range(len(data["product_name"])):
    for j in range(i + 1, len(data["product_name"])):
        similarity_pairs.append(
            (data["product_name"][i], data["product_name"][j], cosine_matrix[i][j])
        )
        
similarity_pairs.sort(key=lambda x: x[2], reverse=True)
similarity_pairs[:5]

[('Chair', 'Table', 0.25938762953244804),
 ('Ottoman', 'Dresser', 0.25085690793255755),
 ('Chair', 'Sofa', 0.2505512775077498),
 ('Chair', 'Bed', 0.2456180219805501),
 ('Table', 'Dresser', 0.24096224354397955)]